In [2]:
import numpy as np
import pandas as pd
import urllib.request
import os
import re
import sys
from os.path import basename   
import os
from collections import defaultdict
from collections import Counter

class FileProcessor:
    def __init__(self):
        self.indexing = pd.DataFrame()
        self.pdf_file_list = []
        self.txt_file_list = []
    
    # DB로 부터 다운로드
    def read_db_indexing_file(self):
        self.indexing = pd.read_csv("./data/db_indexing_final.csv",encoding='EUC-KR')
        isNan = fc.indexing.iloc[:,2].isnull()
    
        self.indexing = self.indexing.fillna("NULL")
        
        # .pdf찾기 (pdf 아닌것 제거)
        f_list = list(self.indexing["SERVICE_URI"])
        is_pdf = [f.endswith('.pdf') for f in f_list ]
        self.indexing = self.indexing.iloc[is_pdf,:]
        
        self.indexing = self.indexing.reset_index(drop=True)
        
    def download_KEI_reports(self):
        for i in range(self.indexing.shape[0]):
            #print(i)
            url = self.indexing["SERVICE_URI"][i]
            #print(url)
            new_path = "./data/pdf/" + os.path.basename(self.indexing["SERVICE_URI"][i])

            
            # 파일이 있으면 다운로드 하지 않음
            if os.path.isfile(new_path) == False :
                urllib.request.urlretrieve(urllib.request.quote(url.encode('utf8'), '/:'),new_path)
            self.pdf_file_list.append(new_path)
        
        pdf_file_list_df = pd.DataFrame(self.pdf_file_list, columns=["PDF_FILE"])
        self.indexing = pd.concat([self.indexing, pdf_file_list_df],axis=1)
        
        
    def pdf_to_txt(self):      
        save_dir = "./result/txt/"
        for i in range(self.indexing.shape[0]):
            new_file = save_dir + os.path.splitext(basename(self.pdf_file_list[i]))[0] + ".txt"
            if os.path.isfile(new_file) == False :
                print("실행")
                cmd = "pdftotext" + " " + "\"" + self.pdf_file_list[i] + "\"" + " " + "\"" + new_file + "\"" 
                print(cmd)
                os.system(cmd)
            if os.path.isfile(new_file) == True :
                self.txt_file_list.append(new_file)
            else :
                self.txt_file_list.append(np.NAN)
                
        txt_file_list_df = pd.DataFrame(self.txt_file_list,columns=["TXT_FILE"])
        self.indexing = pd.concat([self.indexing, txt_file_list_df],axis=1) 
        isNan = self.indexing.iloc[:,5].isnull()

        self.indexing = self.indexing.dropna()
        self.indexing = self.indexing.reset_index(drop=True)
        print(self.indexing.columns)
        #self.indexing.columns.appand("PDF_PATH")
        #self.indexing.columns.append("TXT_PATH")
        #print(self.indexing)
        
    def write_indexing_file(self):
        self.indexing.to_csv("./result/indexing_final.csv",header=True,index=None)
   
        

In [3]:
fc = FileProcessor()
fc.read_db_indexing_file()
fc.download_KEI_reports()
fc.pdf_to_txt()
fc.write_indexing_file()

FileNotFoundError: [Errno 2] No such file or directory: './data/pdf/[95_정책]WO-04환경예산정책(곽태원).pdf'

In [42]:
# wed : Web Extraction from Docs
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import os
import re
import sys
from os.path import basename   
import os
from collections import defaultdict
from collections import Counter
import matplotlib.pylab as plt
%matplotlib inline

class WebSiteExtracter : 
    
    def __init__ (self) :
        self.indexing = pd.DataFrame()
        self.website_list = []
        self.website_df = pd.DataFrame()
        self.category_df = None
    def read_indexing_file(self):
        self.indexing = pd.read_csv("./result/indexing.csv")
        self.indexing["KEI_RESEARCHAREA"] = self.indexing["KEI_RESEARCHAREA"].fillna("미분류")
        
    def calculate_statistics(self):
        category_cnt = Counter(self.indexing["KEI_RESEARCHAREA"].tolist())
        category_df = defaultdict(lambda: 0)
        category_df['자원순환'] = category_cnt['자원순환관리'] + category_cnt['자원순환'] +  category_cnt['폐기물관리']
        category_df['기후변화'] = category_cnt['기후변화'] + category_cnt['기후변화 대응'] +  category_cnt['기후변화대응']
        category_df['자연환경'] = category_cnt['국토환경관리'] + category_cnt['자연환경'] 
        category_df['환경정책'] = category_cnt['환경정책'] + category_cnt['정책일반'] 
        category_df['환경영향평가'] = category_cnt['환경영향평가'] + category_cnt['환경평가'] 
        category_df['지구환경'] = category_cnt['지구환경'] + category_cnt['국제협력'] 
        category_df['대기환경'] = category_cnt['기후변화대기'] + category_cnt['대기환경'] 
        category_df['기타'] = category_cnt['기타'] + category_cnt['위해성관리'] + category_cnt['환경경제']
        category_df['물환경'] = category_cnt['물환경']
        category_df['환경보건'] = category_cnt['환경보건']
        category_df['미분류'] = category_cnt['미분류']
        self.category_df = category_df

        
    def extract_website(self):
        website_list_cnt = defaultdict(lambda: 0)
        category_list_cnt = defaultdict(lambda: 0)   
        ko_keyword_list = defaultdict(lambda: "")
        en_keyword_list = defaultdict(lambda: "")
        
        
        categories = np.array(list(set(self.indexing["KEI_RESEARCHAREA"])))
        #print(categories)
        
        for i in range(self.indexing.shape[0]):
            #print(file)
            file = self.indexing['TXT_FILE'][i]
            category = self.indexing['KEI_RESEARCHAREA'][i]
            ko_keyword = self.indexing['KEI_KEYWORDKOREA'][i]
            #print(ko_keyword)
            en_keyword = self.indexing['KEI_KEYWORDENGLISH'][i]
            #print(en_keyword)
            
            website_list = []
            
            #print(file)
            f = open(file, 'r',encoding='utf-8',errors="ignore")
            doc = f.read()
        
            p = re.compile('([--:\w?%&+~#=]*\.[a-z]{2,4}\/{0,2})((?:[?&](?:\w+)=(?:\w+))+|[--:\w?@%&+~#=]+)?/g')
            w_list = p.findall(doc)
            if(len(w_list) == 0): next
                
            # 한문서 내에서는 1번만 사이트 카운트
            w_list = list(set(w_list))
            
            # 제일앞에 아무것도 없는 경우는 http를 붙힘
            for w in w_list:           
                w = "".join(w)
                if (w.startswith("http://") == False and 
                    w.startswith("https://") == False and 
                    w.startswith("ftp://") == False):
                    w = "http://" + w
                #print(w)
                website_list.append(w)     
                
            f.close()
        
            for w in website_list:
                if (w.startswith("https://")):
                    fstr = "http://" + w[8:]
                    #print(fstr)
                    if fstr in website_list:
                        website_list.remove(fstr)
                        
            for w in website_list:
                try : 
                    website_list_cnt[w] += 1
                    ko_keyword_list[w] += "," + ko_keyword
                    en_keyword_list[w] += "," + en_keyword
                    category_label = (categories == category)
                    category_label = category_label.astype(int)
                    category_label = category_label.tolist()
                    category_list_cnt[w] =  (np.array(category_list_cnt[w]) + np.array(category_label)).tolist()                    
                    
                except : 
                    website_list_cnt[w] = 1 
                    category_label = (categories == category)
                    category_label = category_label.astype(int)
                    category_label = category_label.tolist()
                    category_list_cnt[w] = category_label

        #print(ko_keyword_list)
                    
        # https주소에 대해서 http가 존재하면 http주소는 없애고 https에 카운트를 더함
        
        #print(website_list_cnt)
        keys = list(website_list_cnt.keys())
        for w in keys:
            if (w.startswith("https://")):
                fstr = "http://" + w[8:]                
                if fstr in keys:
                    cnt = website_list_cnt[fstr]
                    del website_list_cnt[fstr]
                    category_cnt = category_list_cnt[fstr]
                    del category_list_cnt[fstr]
                    
                    website_list_cnt[w] = website_list_cnt[w] + cnt
                    category_list_cnt[w] =  (np.array(category_list_cnt[w]) + np.array(category_cnt)).tolist()
                    try:
                        ko_keyword = ko_keyword_list[fstr] 
                        del ko_keyword_list[fstr] 
                        ko_keyword_list[w] += "," + str(ko_keyword)
                    except:
                        ko_keyword_list[w] = ""
                        
                    try:
                        en_keyword = en_keyword_list[fstr] 
                        del en_keyword_list[fstr] 
                        en_keyword_list[w] += "," + str(en_keyword)
                    except:
                        en_keyword_list[w] = ""
                    
        #print(ko_keyword_list)
        website_df = pd.DataFrame(list(website_list_cnt.items()))    
        category_df = pd.DataFrame(list(category_list_cnt.items()))
        
        colnames = ["Website","Cnt","Website2"]
    
        colnames = list(colnames) + list(categories)
               
        result_df = pd.DataFrame(columns = colnames)
        
        result_df[['Website','Cnt']] = website_df
        result_df['Website2'] = category_df.iloc[:,0]
         
        temp = list(category_df.iloc[:,1])
        temp = list(zip(*temp))
        
        for j in range(len(categories)):
            result_df[categories[j]] = temp[j]
                      
        self.website_df = result_df
        self.website_df.columns = colnames
        self.website_df = self.website_df.sort_values("Cnt",ascending=False)
        self.website_df = self.website_df.reset_index()
        
        title_list = []
        # add_title
        # for i in range(website_df.count()[0]):
        for i in range(10):
            #print(i)
            url = self.website_df["Website"][i]
            try :
                cur_html = requests.get(url).text
                soup = BeautifulSoup(cur_html, 'lxml')   
                title = soup.select('head > title')
                title = title[0].text
                title_list.append(title)
            except : 
                title_list.append("Error")
        title_list = pd.DataFrame(title_list,columns=["타이틀"])
        self.website_df = pd.concat([self.website_df ,title_list],axis=1)
        
        # 카테고리 제 분류
        self.website_df['자원순환'] = self.website_df['자원순환관리'] + self.website_df['자원순환'] +  self.website_df['폐기물관리']
        self.website_df['기후변화'] = self.website_df['기후변화'] + self.website_df['기후변화 대응'] +  self.website_df['기후변화대응']
        self.website_df['자연환경'] = self.website_df['국토환경관리'] + self.website_df['자연환경'] 
        self.website_df['환경정책'] = self.website_df['환경정책'] + self.website_df['정책일반'] 
        self.website_df['환경영향평가'] = self.website_df['환경영향평가'] + self.website_df['환경평가'] 
        self.website_df['지구환경'] = self.website_df['지구환경'] + self.website_df['국제협력'] 
        self.website_df['대기환경'] = self.website_df['기후변화대기'] + self.website_df['대기환경'] 
        self.website_df['기타'] = self.website_df['위해성관리'] + self.website_df['환경경제'] 
        
        self.website_df = self.website_df.drop('자원순환관리',axis=1)
        self.website_df = self.website_df.drop('폐기물관리',axis=1)
        self.website_df = self.website_df.drop('기후변화 대응',axis=1)
        self.website_df = self.website_df.drop('기후변화대응',axis=1)
        self.website_df = self.website_df.drop('국토환경관리',axis=1)
        self.website_df = self.website_df.drop('정책일반',axis=1)
        self.website_df = self.website_df.drop('환경평가',axis=1)
        self.website_df = self.website_df.drop('국제협력',axis=1)
        self.website_df = self.website_df.drop('기후변화대기',axis=1)
        self.website_df = self.website_df.drop('위해성관리',axis=1)
        self.website_df = self.website_df.drop('환경경제',axis=1)        
        del self.website_df["Website2"]
        
        ko_keyword_list.head(3)
        en_keyword_list.head(3)
        
        self.website_df['KR_Keyword'] = ko_keyword_list
        self.website_df['EN_Keyword'] = en_keyword_list
        #print(self.website_df)
        
   
        
    def write_result(self):
        self.website_df.to_csv("./result/result.csv",index=None)
    


In [43]:
we = WebSiteExtracter()
we.read_indexing_file()
we.extract_website()
#plt.hist(we.indexing["KEI_RESEARCHAREA"].tolist())
#we.calculate_statistics()
#we.website_df
#we.write_result()
#we.indexing["category"]

AttributeError: 'collections.defaultdict' object has no attribute 'head'

In [ ]:
we.website_df